# TP3 : Branch-and-Cut

In [2]:
import cplex
from itertools import combinations


## Question 1 : Modèle sans contrainte (4)


In [4]:
def lire_instance(fichier):
    with open(fichier, "r") as f:
        n = int(f.readline().strip())
        C = []
        for _ in range(n):
            C.extend(map(float, f.readline().strip().split()))
    return n, C

def construire_modele(n, C):
    model = cplex.Cplex()
    model.set_problem_type(cplex.Cplex.problem_type.LP)
    model.objective.set_sense(model.objective.sense.minimize)

    x_names = [f"x_{i}_{j}" for i in range(n) for j in range(n)]
    model.variables.add(
        obj=C,
        names=x_names,
        lb=[0.0] * len(x_names),
        ub=[1.0] * len(x_names),
        types=["B"] * len(x_names)
    )

    for j in range(n):
        entree = [f"x_{i}_{j}" for i in range(n) if i != j]
        model.linear_constraints.add(
            lin_expr=[cplex.SparsePair(entree, [1] * len(entree))],
            senses=["E"],
            rhs=[1]
        )

    for i in range(n):
        sortie = [f"x_{i}_{j}" for j in range(n) if i != j]
        model.linear_constraints.add(
            lin_expr=[cplex.SparsePair(sortie, [1] * len(sortie))],
            senses=["E"],
            rhs=[1]
        )

    return model, x_names



## Question 2 : Exécution sur tsp_small.txt


In [6]:
fichier_small = "tsp_small.txt"
n_small, C_small = lire_instance(fichier_small)
model_small, x_names_small = construire_modele(n_small, C_small)
model_small.write("tsp_model.lp")
model_small.solve()

valeurs = model_small.solution.get_values()
selected_arcs = [(i, j) for i in range(n_small) for j in range(n_small) if model_small.solution.get_values(f"x_{i}_{j}") > 0.5]
print("\n=== Résultat Question 2 ===")
print("Statut:", model_small.solution.get_status_string())
print("Coût:", model_small.solution.get_objective_value())
print("Arcs sélectionnés:", selected_arcs) 

Default row names c1, c2 ... being created.


Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 5 columns.
Reduced MIP has 10 rows, 20 columns, and 40 nonzeros.
Reduced MIP has 20 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.02 ticks)
Found incumbent of value 957.000000 after 0.01 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 10 rows, 20 columns, and 40 nonzeros.
Reduced MIP has 20 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 10.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.02 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf 

## Question 3 : Ajout de la contrainte (4) sur tsp_small.txt


In [8]:
def detect_subtours(arcs, n):
    unvisited = set(range(n))
    subtours = []
    while unvisited:
        current = unvisited.pop()
        tour = [current]
        while True:
            next_node = None
            for i, j in arcs:
                if i == current:
                    next_node = j
                    break
            if next_node is None or next_node in tour:
                break
            tour.append(next_node)
            unvisited.discard(next_node)
            current = next_node
        subtours.append(tour)
    return subtours

def resoudre_et_ajouter_contraintes(model, x_names, n):
    while True:
        model.solve()
        valeurs = model.solution.get_values()
        arcs = []
        for k in range(len(valeurs)):
            if valeurs[k] > 0.5:
                name = x_names[k]
                i, j = map(int, name.split("_")[1:])
                arcs.append((i, j))

        subtours = detect_subtours(arcs, n)
        print("Sous-tours détectés:", subtours)

        if len(subtours) == 1 and len(subtours[0]) == n:
            return arcs, subtours[0]

        for S in subtours:
            if len(S) < n:
                coupe_vars = [f"x_{i}_{j}" for i in S for j in S if i != j]
                model.linear_constraints.add(
                    lin_expr=[cplex.SparsePair(coupe_vars, [1.0] * len(coupe_vars))],
                    senses=["L"],
                    rhs=[len(S) - 1]
                )

def reconstruire_tournee(arcs, n):
    tournee = [0]
    courant = 0
    while len(tournee) < n:
        for (i, j) in arcs:
            if i == courant:
                tournee.append(j)
                courant = j
                break
    tournee.append(0)
    return tournee

arcs_solution, cycle = resoudre_et_ajouter_contraintes(model_small, x_names_small, n_small)
tournee_finale = reconstruire_tournee(arcs_solution, n_small)
print("\n=== Résultat Question 3 ===")
print("Tournée optimale:", tournee_finale)


Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.00 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.00 ticks)
Sous-tours détectés: [[0, 3], [1, 4, 2]]
Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1
1 of 2 MIP starts provided solutions.
MIP start 'm2' defined initial solution with objective 957.0000.
Retaining values of one MIP start for possible repair.
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 5 columns.
Reduced MIP has 12 rows, 20 columns, and 48 nonzeros.
Reduced MIP has 20 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (

## Question 4 : Application à tsp_large.txt

In [10]:
fichier_large = "tsp_large.txt"
n_large, C_large = lire_instance(fichier_large)
model_large, x_names_large = construire_modele(n_large, C_large)
arcs_solution_large, cycle_large = resoudre_et_ajouter_contraintes(model_large, x_names_large, n_large)
tournee_large = reconstruire_tournee(arcs_solution_large, n_large)
print("\n=== Résultat Question 4 ===")
print("Tournée optimale (tsp_large):", tournee_large)


Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 4288.000000 after 0.00 sec. (0.03 ticks)
Found incumbent of value 1740.000000 after 0.00 sec. (0.04 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 20 columns.
Reduced MIP has 40 rows, 380 columns, and 760 nonzeros.
Reduced MIP has 380 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.46 ticks)
Probing time = 0.00 sec. (0.70 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 40 rows, 380 columns, and 760 nonzeros.
Reduced MIP has 380 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.60 ticks)
Probing time = 0.02 sec. (0.70 ticks)
Clique table members: 40.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.21 ticks)

        Nodes       

Retaining values of one MIP start for possible repair.
Tried aggregator 1 time.
MIP Presolve eliminated 0 rows and 20 columns.
Reduced MIP has 48 rows, 380 columns, and 792 nonzeros.
Reduced MIP has 380 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.75 ticks)
Found incumbent of value 5346.000000 after 0.02 sec. (1.95 ticks)
Probing time = 0.00 sec. (0.72 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 48 rows, 380 columns, and 792 nonzeros.
Reduced MIP has 380 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.89 ticks)
Probing time = 0.00 sec. (0.72 ticks)
Clique table members: 44.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.32 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     G